<img src="https://github.com/Multiomics-Analytics-Group/course_protein_language_modeling/blob/main/img/nb_logo.png?raw=1" width="600">

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Multiomics-Analytics-Group/course_protein_language_modeling/blob/main/notebooks/seq_analysis.ipynb)

# Introduction to BioPython

[BioPython](https://biopython.org/) is a set of freely available tools for biological computation written in Python by an international team of developers. It is a distributed collaborative effort part of the Open Bioinformatics Foundation (OBF).

This notebook was inspired by the BioPython notebooks provided [here](https://github.com/tiagoantao/biopython-notebook).

In [ ]:
!pip install biopython

## Sequence Objects

Biopython deals with sequences using the Seq object. Sequences are essentially strings of letters like AGTACACTGGT, but the difference between Seq objects and standard Python strings is that they have different methods to perform biologically relevant methods.

In [ ]:
from Bio.Seq import Seq

my_seq = Seq("GATCG")

for index, letter in enumerate(my_seq):
    print("%i %s" % (index, letter))

print(len(my_seq))

You can access elements of the sequence in the same way as for strings (a list of characters)

In [ ]:
print(my_seq[0]) #first nucleotide
print(my_seq[2]) #third nucleotide
print(my_seq[-1]) #last nucleotide

The Seq object has a .count() method, just like a string.
Note that this means that like a Python string, this gives a
non-overlapping count:


In [ ]:
print("AAAA".count("A"))
print(Seq("AAAA").count("AA"))

For some biological uses, you may actually want an overlapping count
(i.e. $3$ in this trivial example). When searching for single letters, this
makes no difference:

In [ ]:
my_seq = Seq('GATCGATGGGCCTATATAGGATCGAAAATCGC')
print(len(my_seq))
print(my_seq.count("G"))
print(100 * float(my_seq.count("G") + my_seq.count("C")) / len(my_seq))

While you could use the above snippet of code to calculate a GC\%, note that  the __Bio.SeqUtils__ module has several GC functions already built.  For example:


In [ ]:
 from Bio.SeqUtils import gc_fraction

gc_fraction(my_seq)

## Turning Seq objects into strings

If you really do just need a plain string, for example to write to a file, or insert into a database, then this is very easy to get:

In [ ]:
str(my_seq)

You can also use the Seq object directly with a %s placeholder when using the Python string formatting or interpolation operator (%):

In [ ]:
fasta_format_string = ">Name\n%s\n" % my_seq
print(fasta_format_string)

## Transcription and Translation

Biological transcription works from the template strand, doing a reverse complement (TACC -> GGTA) to give the mRNA.  However, in Biopython we typically work directly with the coding strand because this means we can get the mRNA sequence just by switching (TACC -> UACC).


|    | DNA coding strand (Crick strand, strand +1)    |    |
| -- | ---------------------------------------------- | -- |
| 5' | ATGGCCATTGTAATGGGCCGCTGAAAGGGTGCCCGATAG        | 3' |
| 3' | TACCGGTAACATTACCCGGCGACTTTCCCACGGGCTATC        | 5' |
|    | DNA template strand (Watson strand, strand -1) |    |
|    |           __Transcription__                    |    |
| 5' | AUGGCCAUUGUAAUGGGCCGCUGAAAGGGUGCCCGAUAG}       | 3' |
|    | Single stranded messenger RNA                  |    |


In [ ]:
coding_dna = Seq("ATGGCCATTGTAATGGGCCGCTGAAAGGGTGCCCGATAG")
print(coding_dna)
template_dna = coding_dna.reverse_complement()
print(template_dna)

These should match the figure above - remember by convention nucleotide sequences are normally read from the 5' to 3' direction, while in the figure the template strand is shown reversed.

Now let's transcribe the coding strand into the corresponding mRNA, using the Seq object's built in transcribe method:

In [ ]:
messenger_rna = coding_dna.transcribe()
messenger_rna

## Translation

Sticking with the same example discussed in the transcription section above,
now let's translate this mRNA into the corresponding protein sequence - again taking
advantage of one of the Seq object's biological methods:

In [ ]:
messenger_rna.translate()

You can also translate directly from the coding strand DNA sequence:

In [ ]:
coding_dna.translate()

You should notice in the above protein sequences that in addition to the end stop character (__*__), there is an internal stop as well.  This is because we are using the default genetic code (NCBI's standard genetic code - table 1), but there are other translation tables (Genetic Codes) that we can use. Suppose we are instead dealing with a Mitochondrial sequence. We need to tell the translation function to use the relevant genetic code instead:

In [ ]:
coding_dna.translate(table="Vertebrate Mitochondrial")

You can also specify the table using the NCBI table number which is shorter, and often included in the feature annotation of GenBank files:


In [ ]:
coding_dna.translate(table=2)

Now, you may want to translate the nucleotides up to the first in frame stop codon,
and then stop (as happens in nature):

In [ ]:
coding_dna.translate(to_stop=True)

Notice that when you use the to_stop argument, the stop codon itself
is not translated - and the stop symbol is not included at the end of your protein
sequence.

You can even specify the stop symbol if you don't like the default asterisk:

In [ ]:
coding_dna.translate(table=2, stop_symbol="@")

# Sequence annotation objects

The Sequence Record or [SeqRecord class](http://biopython.org/DIST/docs/api/Bio.SeqRecord.SeqRecord-class.html), defined in the [Bio.SeqRecord module](http://biopython.org/DIST/docs/api/Bio.SeqFeature.SeqFeature-class.html) allows higher level features such as identifiers and features (as SeqFeature objects) to be associated with the sequence, and is used throughout the sequence input/output interface [Bio.SeqIO](https://biopython.org/docs/1.75/api/Bio.SeqIO.html?highlight=seqio#module-Bio.SeqIO) to read and write FASTA files.

In [ ]:
from Bio.SeqRecord import SeqRecord

## The SeqRecord Object

The SeqRecord (Sequence Record) class is defined in the Bio.SeqRecord module. This class allows higher level features such as identifiers and features to be associated with a sequence, and is the basic data type for the Bio.SeqIO sequence input/output interface.

The SeqRecord class itself is quite simple, and offers the following information as attributes:

*  **.seq** - The sequence itself, typically a Seq object.

* **.id** - The primary ID used to identify the sequence - a string. In most cases this is something like an accession number.

* **.name** - A 'common' name/id for the sequence - a string. In some cases this will be the same as the accession number, but it could also be a clone name. I think of this as being analogous to the LOCUS id in a GenBank record.

* **.description** - A human readable description or expressive name for the sequence - a string.
  
* **.letter_annotations** - Holds per-letter-annotations using a (restricted) dictionary of additional information about the letters in the sequence. The keys are the name of the information, and the information is contained in the value as a Python sequence (i.e. a list, tuple or string) with the same length as the sequence itself.  This is often used for quality scores or secondary structure information (e.g. from Stockholm/PFAM alignment files).

* **.annotations** - A dictionary of additional information about the sequence. The keys are the name of the information, and the information is contained in the value. This allows the addition of more 'unstructured' information to the sequence.
  
* **.features** - A list of SeqFeature objects with more structured information about the features on a sequence (e.g. position of genes on a genome, or domains on a protein sequence).
  
* **.dbxrefs** - A list of database cross-references as strings.


## Creating a SeqRecord

Using a SeqRecord object is not very complicated, since all of the
information is presented as attributes of the class. Usually you won't create
a SeqRecord 'by hand', but instead use Bio.SeqIO to read in a
sequence file for you and the examples
below).  However, creating SeqRecord can be quite simple.

### SeqRecord objects from scratch

To create a SeqRecord at a minimum you just need a Seq object:

In [ ]:
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq

simple_seq = Seq("GATC")
simple_seq_r = SeqRecord(simple_seq)

Additionally, you can also pass the id, name and description to the initialization function, but if not they will be set as strings indicating they are unknown, and can be modified subsequently:

In [ ]:
simple_seq_r.id

In [ ]:
simple_seq_r.id = "AC12345"
simple_seq_r.description = "Not a real sequence"
print(simple_seq_r.description)
simple_seq_r.seq
print(simple_seq_r.seq)

Including an identifier is very important if you want to output your SeqRecord to a file.  You would normally include this when creating the object:


In [ ]:
simple_seq = Seq("GATC")
simple_seq_r = SeqRecord(simple_seq, id="AC12345")

As mentioned above, the SeqRecord has an dictionary attribute annotations. This is used
for any miscellaneous annotations that doesn't fit under one of the other more specific attributes.
Adding annotations is easy, and just involves dealing directly with the annotation dictionary:

In [ ]:
simple_seq_r.annotations["evidence"] = "None"
print(simple_seq_r.annotations)
print(simple_seq_r.annotations["evidence"])

Working with per-letter-annotations is similar, letter_annotations is a
dictionary like attribute which will let you assign any Python sequence (i.e.
a string, list or tuple) which has the same length as the sequence:

In [ ]:
simple_seq_r.letter_annotations["phred_quality"] = [40, 40, 38, 30]
print(simple_seq_r.letter_annotations)
print(simple_seq_r.letter_annotations["phred_quality"])

The dbxrefs and features attributes are just Python lists, and
should be used to store strings and SeqFeature objects (discussed later) respectively.

## Obtain and Parse Sequences

Here, we will download and read a FASTA file containing the sequence of a list of genes from \textit{Escherichia coli}. We will make use of the [Entrez module](https://biopython.org/docs/1.76/api/Bio.Entrez.html). This module provides a number of functions like `efetch` (short for Entrez Fetch) which will return the searched data as a handle object (a file-like object).

### Getting Nucleotide Sequences

In [ ]:
from Bio import SeqIO
from Bio import Entrez

# The line below carries out a search of the `assembly` database at NCBI,
# using the phrase `Ralstonia solanacearum` as the search query,
# and asks NCBI to return up to the first 100 results
with Entrez.esearch(db="assembly", term="Escherichial coli", retmax=10) as handle:
    # This line converts the returned information from NCBI into a form we
    # can use, as before.
    record = Entrez.read(handle)
print(record)

In [ ]:
Entrez.email = 'A.N.Other@example.com'
with Entrez.efetch(db="nucleotide", id=record["IdList"], rettype="gb", retmode="text") as handle:
    record_iterator = SeqIO.parse(handle, "gb")

    for record in record_iterator:
        print(record.format("fasta"))

### Getting Protein Sequences

In [ ]:
with Entrez.esearch(db="protein", term="Escherichial coli", retmax=10) as handle:
    record = Entrez.read(handle)
print(record)

In [ ]:
with Entrez.efetch(db="protein", id=record["IdList"], rettype="gb", retmode="text") as handle:
    record_iterator = SeqIO.parse(handle, "gb")
    
    for record in record_iterator:
        print(record.format("fasta"))

# Protein Parameters

When working with sequences, especially in ML models, it is important to extract some parameters that summarize them and that can be used for instance to make sure that your train/test/validation splits are comparable.

We can make use of `Bio.SeqUtils.ProtParam` to extract these parameters and visualize them.

In [ ]:
from Bio.SeqUtils.ProtParam import ProteinAnalysis


protein_sequence = ProteinAnalysis("MAEGEITTFTALTEKFNLPPGNYKKPKLLYCSNGGHFLRILPDGTVDGT"
                    "RDRSDQHIQLQLSAESVGEVYIKSTETGQYLAMDTSGLLYGSQTPSEEC"
                    "LFLERLEENHYNTYTSKKHAEKNWFVGLKKNGSCKRGPRTHYGQKAILF"
                    "LPLPV")

print(protein_sequence.count_amino_acids())

print("%0.2f" % protein_sequence.molecular_weight())

print("%0.2f" % protein_sequence.aromaticity())

print("%0.2f" % protein_sequence.instability_index())

print("%0.2f" % protein_sequence.isoelectric_point())

sec_struc = protein_sequence.secondary_structure_fraction()  # [helix, turn, sheet]
print(sec_struc)

epsilon_prot = protein_sequence.molar_extinction_coefficient()  # [reduced, oxidized]
print(epsilon_prot[0])  # with reduced cysteines
print(epsilon_prot[1])  # with disulfid bridges

In [ ]:
train_sequences = ['MIQEKDKYVVASVTILESNQILDKGSIQTSGIELNYYMKYQKY',
             'MIQEKDKYVVASVTILESNQLLKEANSSGN',
             'MIQEKDKYVVASVTILESNQNSDVFIDLTNLTVEHKKLLN', 
             'MIQEKDKYVVASVTILESNQVFDAMGFGGFLNINRKKFYNDL',
             'MIQEKDKYVVASVTILESNQVFFDSPEIKKLGLVTLKAKYD',
             'MIQEKDKYVVASVTILESNQFGMKRNILYKETKALRAK']
test_sequences = ['MIQEKDKYVVASVTILESNQILDKGSIQAAAALNYYMKYQKY',
             'MIQEKDKAAAAHHHHHHTILESNQLLKEANSSGN',
             'MIQEAAAAAASVTILHHHHHHHHESNQNSDVFIDLTNLTVEHKKLLN', 
             'MIQEKDKYVVLLLLLLLSSSSSAAAAAAGFGGFLNINRKKFYNDL',
             'MIQEKDKYVVASVTILESNQVFFDSSSSSAAAAAAAKKLGLVTLKAKYD',
             'MIQEKDKYVVASVTILESNQFGMKRNILYKLLLLSSSSSSSSSKKKKKAAAAAAK']

In [ ]:
import pandas as pd

def calculate_mean_AA_composition(sequences):
    stats = []
    for sequence in sequences:
        protein_sequence = ProteinAnalysis(sequence)
        st = {k:v/len(sequence) for k,v in protein_sequence.count_amino_acids().items()}
        stats.append(st)
    df = pd.DataFrame.from_dict(stats).transpose()
    df = df.T.mean().reset_index()
    df.columns = ['amino acid', 'mean']

    return df

In [ ]:
train_df = calculate_mean_AA_composition(train_sequences)
test_df = calculate_mean_AA_composition(test_sequences)

In [ ]:
import plotly.express as px
import plotly.io as pio
pio.renderers.default = 'colab'

fig = px.bar(train_df.sort_values(by='mean'), x='amino acid', y='mean', color='mean')
fig.show()

In [ ]:
fig = px.bar(test_df.sort_values('mean'), x='amino acid', y='mean', color='mean')
fig.show()

## Multiple Sequence Alignment objects

We will now look at collections of sequences which have been aligned together –
usually with the insertion of gap characters, and addition of leading or
trailing gaps – such that all the sequence strings are the same length.
Such an alignment can be regarded as a matrix of letters, where each row
is held as a `SeqRecord` object internally.

We will introduce the `MultipleSeqAlignment` object which holds this
kind of data, and the `Bio.AlignIO` module for reading and writing them
as various file formats (following the design of the `Bio.SeqIO` module).

### Parsing or Reading Sequence Alignments

There are two functions for reading in sequence alignments,
`Bio.AlignIO.read()` and `Bio.AlignIO.parse()` which following the
convention introduced in `Bio.SeqIO` are for files containing one or
multiple alignments respectively.

Using `Bio.AlignIO.parse()` will return an <span>*iterator*</span> which
gives `MultipleSeqAlignment` objects.

However, in many situations you will be dealing with files which contain
only a single alignment. In this case, you should use the
`Bio.AlignIO.read()` function which returns a single
`MultipleSeqAlignment` object.

Both functions expect two mandatory arguments:

1.  The first argument is a <span>*handle*</span> to read the data from,
    typically an open file (see Section \[sec:appendix-handles\]), or
    a filename.

2.  The second argument is a lower case string specifying the
    alignment format. As in `Bio.SeqIO` we don’t try and guess the file
    format for you! See [AlignIO](http://biopython.org/wiki/AlignIO) for a full
    listing of supported formats.

A further optional `alphabet` argument allowing you to specify the
expected alphabet. This can be useful as many alignment file formats do
not explicitly label the sequences as RNA, DNA or protein – which means
`Bio.AlignIO` will default to using a generic alphabet.

In [ ]:
!wget -q https://raw.githubusercontent.com/Multiomics-Analytics-Group/course_protein_language_modeling/main/data/Test_seed1_aligned.aln -O Test_seed1_aligned.aln

In [ ]:
!pip install pymsaviz

In [ ]:
from Bio import AlignIO
alignment = AlignIO.read("Test_seed1_aligned.aln", format="fasta")


This code will print out a summary of the alignment:



In [ ]:
print(alignment)

In [ ]:
from pymsaviz import MsaViz, get_msa_testdata

mv = MsaViz(alignment)
fig = mv.plotfig()

# Protein Structures

Bio.PDB is a Biopython module that focuses on working with crystal
structures of biological macromolecules. Among other things, Bio.PDB
includes a PDBParser class that produces a Structure object, which can
be used to access the atomic data in the file in a convenient manner.
There is limited support for parsing the information contained in the
PDB header.

Reading and writing crystal structure files
-------------------------------------------

### Reading a PDB file

First we create a `PDBParser` object:

In [ ]:
from Bio.PDB.PDBParser import PDBParser
p = PDBParser(PERMISSIVE=1)


The <span>PERMISSIVE</span> flag indicates that a number of common
problems (see \[problem structures\]) associated with PDB files will be
ignored (but note that some atoms and/or residues will be missing). If
the flag is not present a <span>PDBConstructionException</span> will be
generated if any problems are detected during the parse operation.

The Structure object is then produced by letting the `PDBParser` object
parse a PDB file (the PDB file in this case is called ’pdb1fat.ent’,
’1fat’ is a user defined name for the structure):

In [ ]:
from Bio.PDB.PDBList import PDBList

structure_id = "1fat"
pdblist = PDBList()
pdb_file = pdblist.retrieve_pdb_file(pdb_code=structure_id, obsolete=False, pdir='.', file_format='pdb', overwrite=False)

structure = p.get_structure(structure_id, pdb_file)

In [ ]:
structure.header

### Visualizing the Structure

In [ ]:
!pip install py3Dmol

In [ ]:
import py3Dmol

#First we assign the py3Dmol.view as view
view=py3Dmol.view()
#The following lines are used to add the addModel class
#to read the PDB files of chain B and C
view.addModel(open(pdb_file, 'r').read(),'pdb')
#Zooming into all visualized structures 
view.zoomTo()
#Here we set the background color as white
view.setBackgroundColor('white')
#Here we set the visualization style for chain B and C
view.setStyle({'chain':'A'}, {'cartoon': {'color':'yellow'}})
view.setStyle({'chain':'B'}, {'cartoon': {'color':'purple'}})
view.setStyle({'chain':'C'}, {'cartoon': {'color':'blue'}})
view.setStyle({'chain':'D'}, {'cartoon': {'color':'grey'}})
#And we finally visualize the structures using the command below
view.show()